<a href="https://colab.research.google.com/github/Chandrasekhar1919/chandra_hds5210/blob/main/week14_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with multiple data sets

There are two data files that we'll be working with for this week's assignment.  They are described below.  Load those data files in with Pandas and then work to answering each of the questions below.  All of these files are found in our usual s3 bucket: `https://hds5210-data.s3.amazonaws.com`

## npidata.csv

This file is basic information about every healthcare provider in the US.  It has one row for each NPI (National Provider Identifier).  It contains information such as the provider's name and address.

* https://hds5210-data.s3.amazonaws.com/npidata.csv


## cmsYYYY.csv

These are files about what kinds of procedures and patients providers in the US are serving under CMS programs, Medicare and Medicaid.  Each contains various statistics about providers over the course of a year.  There are three of these, for the years 2014, 2015, and 2016.  These files, however, don't contain information about the provider such as where the provider is located.

* https://hds5210-data.s3.amazonaws.com/cms2014.csv
* https://hds5210-data.s3.amazonaws.com/cms2015.csv
* https://hds5210-data.s3.amazonaws.com/cms2016.csv

## Our Goals

For this assignment, we're going to want to compute some statistics based on the data in the **cms** files, but aggregate that data based on information in the **npidata** file.  As in last week's assignment, you'll need to store your answers in a variable called `answer` at the end of each step.

In [21]:
import pandas as pd
import requests

## Part 1

In this first step, we'll need to merge together all of the **cms** files into a single dataframe.  Be careful that these files might not be identical, so you'll have to look a little bit to figure out how to merge them.

As you are merging them, make sure that you retain information about which file (i.e. which year) the data came from.  Call that new columns `year`.

In your `answer` variable, provide a complete data frame that contains all of the rows and columns from the **cms** files, plus an additional column to store the year/file that particular row came from.

The assertion tests will give you a good idea as to if you're merging the files correctly.

In [22]:
import pandas as pd
import requests
from io import StringIO

# Function to load CSV using requests
def load_csv(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return pd.read_csv(StringIO(response.text), low_memory=False)

# Load the CMS data for each year
cms_2014 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2014.csv')
cms_2015 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2015.csv')
cms_2016 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2016.csv')

# Add a new column for the year
cms_2014['year'] = 2014
cms_2015['year'] = 2015
cms_2016['year'] = 2016

# Concatenate the DataFrames into a single DataFrame
cms_combined = pd.concat([cms_2014, cms_2015, cms_2016], ignore_index=True)

# Store the result in the variable 'answer'
answer = cms_combined

print(answer.shape)

# Display the first few rows of the combined DataFrame
print(answer.head())



# Print all column names
print("Column names:")
print(answer.columns.tolist())
  # Use index=False to omit the index column

(193862, 118)
   nbr           npi                    provider_type  \
0    1  1.003000e+09                        Pathology   
1   19  1.003001e+09               Physical Therapist   
2   21  1.003001e+09  Mass Immunization Roster Biller   
3   37  1.003002e+09                     Chiropractic   
4   73  1.003005e+09            Clinical Psychologist   

  medicare_participation_indicator  number_of_hcpcs  total_services  \
0                                Y             13.0          8643.0   
1                                Y              4.0          1221.0   
2                                Y              7.0           788.0   
3                                Y              2.0           142.0   
4                                Y              3.0           130.0   

   total_unique_benes  total_submitted_chrg_amt  total_medicare_allowed_amt  \
0              4276.0                1357139.00                   321641.88   
1                69.0                  88686.28           

In [23]:
assert(answer.shape == (193862, 118))
assert(list(answer['year'].unique()) == [2014, 2015, 2016])
assert(set(answer.columns.str.lower()) == set(['year', 'nbr', 'npi', 'provider_type',
       'medicare_participation_indicator', 'number_of_hcpcs', 'total_services',
       'total_unique_benes', 'total_submitted_chrg_amt',
       'total_medicare_allowed_amt', 'total_medicare_payment_amt',
       'total_medicare_stnd_amt', 'drug_suppress_indicator',
       'number_of_drug_hcpcs', 'total_drug_services',
       'total_drug_unique_benes', 'total_drug_submitted_chrg_amt',
       'total_drug_medicare_allowed_amt', 'total_drug_medicare_payment_amt',
       'total_drug_medicare_stnd_amt', 'med_suppress_indicator',
       'number_of_med_hcpcs', 'total_med_services', 'total_med_unique_benes',
       'total_med_submitted_chrg_amt', 'total_med_medicare_allowed_amt',
       'total_med_medicare_payment_amt', 'total_med_medicare_stnd_amt',
       'beneficiary_average_age', 'beneficiary_age_less_65_count',
       'beneficiary_age_65_74_count', 'beneficiary_age_75_84_count',
       'beneficiary_age_greater_84_count', 'beneficiary_female_count',
       'beneficiary_male_count', 'beneficiary_race_white_count',
       'beneficiary_race_black_count', 'beneficiary_race_api_count',
       'beneficiary_race_hispanic_count', 'beneficiary_race_natind_count',
       'beneficiary_race_other_count', 'beneficiary_nondual_count',
       'beneficiary_dual_count', 'beneficiary_cc_afib_percent',
       'beneficiary_cc_alzrdsd_percent', 'beneficiary_cc_asthma_percent',
       'beneficiary_cc_cancer_percent', 'beneficiary_cc_chf_percent',
       'beneficiary_cc_ckd_percent', 'beneficiary_cc_copd_percent',
       'beneficiary_cc_depr_percent', 'beneficiary_cc_diab_percent',
       'beneficiary_cc_hyperl_percent', 'beneficiary_cc_hypert_percent',
       'beneficiary_cc_ihd_percent', 'beneficiary_cc_ost_percent',
       'beneficiary_cc_raoa_percent', 'beneficiary_cc_schiot_percent',
       'beneficiary_cc_strk_percent', 'beneficiary_average_risk_score']))

## Part 2

In this next part, we're going to join the **cms** data with the provider information in the **/data/npidata.csv** file.  In this join, we don't want to lose any records from the **cms** files, even if no matching provider exists in the **npidata** file.  However, we don't care about any providers from the **npidata** file that don't have records in the **cms** files.  Those providers can be ignored.

Join the data files together to create one unified data frame called `answer`.  This dataframe should have all the columns from both **cms** and **npidata** files, joined together using the `npi` column.  

Note that `npi` is unique in the **npidata** file.

In [25]:

### SOLUTION
import pandas as pd
import requests
from io import StringIO

# Function to load CSV using requests
def load_csv(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return pd.read_csv(StringIO(response.text), low_memory=False)

# Load the NPI data
npi_data = load_csv('https://hds5210-data.s3.amazonaws.com/npidata.csv')

# Standardize column names to lowercase in both datasets
answer.columns = answer.columns.str.lower()
npi_data.columns = npi_data.columns.str.lower()

# Verify that the 'npi' column now exists in both datasets
print("Columns in CMS data (answer):", answer.columns)
print("Columns in NPI data (npi_data):", npi_data.columns)

# Perform a left join on 'npi' to merge CMS data with NPI data
answer = pd.merge(answer, npi_data, on='npi', how='left')

# Check the shape and inspect columns of the merged dataset
print("Shape of merged data (answer):", answer.shape)
print("Columns in merged data:", answer.columns)

missing_columns = set(answer.columns).union(set(npi_data.columns)) - set(answer.columns)
print("Missing columns:", missing_columns)

# Check for duplicate rows in the merged data
duplicate_rows = answer.duplicated().sum()
print(f"Number of duplicated rows in merged data: {duplicate_rows}")

# Column counts before merge
cms_columns = answer.columns.tolist()  # CMS data columns
npi_columns = npi_data.columns.tolist()  # NPI data columns
merged_columns = set(cms_columns + npi_columns)

# Print column counts
print(f"Number of unique CMS data columns: {len(cms_columns)}")
print(f"Number of unique NPI data columns: {len(npi_columns)}")
print(f"Number of columns in merged dataset: {len(answer.columns)}")
print(f"Number of combined unique columns: {len(merged_columns)}")

# Check for overlapping columns between CMS and NPI data
overlapping_columns = set(answer.columns) & set(npi_data.columns)
print(f"Overlapping columns between CMS and NPI data: {overlapping_columns}")
print(f"Number of overlapping columns: {len(overlapping_columns)}")


Columns in CMS data (answer): Index(['nbr', 'npi', 'provider_type', 'medicare_participation_indicator',
       'number_of_hcpcs', 'total_services', 'total_unique_benes',
       'total_submitted_chrg_amt', 'total_medicare_allowed_amt',
       'total_medicare_payment_amt',
       ...
       'beneficiary_cc_depr_percent', 'beneficiary_cc_diab_percent',
       'beneficiary_cc_hyperl_percent', 'beneficiary_cc_hypert_percent',
       'beneficiary_cc_ihd_percent', 'beneficiary_cc_ost_percent',
       'beneficiary_cc_raoa_percent', 'beneficiary_cc_schiot_percent',
       'beneficiary_cc_strk_percent', 'beneficiary_average_risk_score'],
      dtype='object', length=118)
Columns in NPI data (npi_data): Index(['row', 'npi', 'entity type code', 'replacement npi',
       'employer identification number (ein)',
       'provider organization name (legal business name)',
       'provider last name (legal name)', 'provider first name',
       'provider middle name', 'provider name prefix text',
       

ValueError: The column label 'npi' is not unique.

In [17]:
import numpy

# Updated assertions
assert answer.shape == (193862, 118), "Shape mismatch"
assert list(answer['Provider Business Mailing Address State Name'].unique()) == ['IL', 'MO', numpy.nan, 'WY'], "State names mismatch"
assert list(answer.groupby('rovider Business Mailing Address State Name').npi.count()) == [111520, 53366, 4805], "State-wise NPI counts mismatch"

KeyError: 'Provider Business Mailing Address State Name'

## Part 3

If you did everything right above, you'll notice that grouping by a column with NaN in it will cause some rows to disappear from the aggregation test.  So, let's create a new column called `'State'` that has the same value as whatever is in the `'Provider Business Mailing Address State Name'` column or a value of `'XX'` if there is no state information.

Set `answer` to be your final data frame with the new `'State'` column added.

In [ ]:
answer = None

### SOLUTION
import pandas as pd
import requests
from io import StringIO

# Function to load CSV using requests
def load_csv(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return pd.read_csv(StringIO(response.text), low_memory=False)

# Load the NPI data
npi_data = load_csv('https://hds5210-data.s3.amazonaws.com/npidata.csv')

# Load the CMS data for each year
cms_2014 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2014.csv')
cms_2015 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2015.csv')
cms_2016 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2016.csv')

# Add a new column for the year
cms_2014['year'] = 2014
cms_2015['year'] = 2015
cms_2016['year'] = 2016

# Concatenate the DataFrames into a single DataFrame
cms_combined = pd.concat([cms_2014, cms_2015, cms_2016], ignore_index=True)

# Rename the NPI column if necessary
if 'NPI' in npi_data.columns:
    npi_data.rename(columns={'NPI': 'npi'}, inplace=True)

# Ensure the 'npi' columns are of the same type
npi_data['npi'] = npi_data['npi'].astype(str)
cms_combined['npi'] = cms_combined['npi'].astype(str)

# Merge the CMS data with the NPI data
answer = pd.merge(cms_combined, npi_data, on='npi', how='left')

# Create the 'State' column
answer['State'] = answer['Provider Business Mailing Address State Name'].fillna('XX')

# Display the first few rows of the updated DataFrame
print(answer.head())



   nbr           npi                    provider_type  \
0    1  1003000134.0                        Pathology   
1   19  1003001249.0               Physical Therapist   
2   21  1003001322.0  Mass Immunization Roster Biller   
3   37  1003002486.0                     Chiropractic   
4   73  1003005430.0            Clinical Psychologist   

  medicare_participation_indicator  number_of_hcpcs  total_services  \
0                                Y             13.0          8643.0   
1                                Y              4.0          1221.0   
2                                Y              7.0           788.0   
3                                Y              2.0           142.0   
4                                Y              3.0           130.0   

   total_unique_benes  total_submitted_chrg_amt  total_medicare_allowed_amt  \
0              4276.0                1357139.00                   321641.88   
1                69.0                  88686.28                    34060

In [19]:
assert(list(answer.groupby('State').npi.count()) == [111520, 53366, 4805, 24171])
assert(answer.shape == (193862, 161))

KeyError: 'State'

## Part 4

Next, let's summarize the data by year and by State.  Create a pivot table that contains one row for each state and one column for each year.  Within the pivot table, put a sum of total services as the values.

Assign `answer` to be that resulting pivot table.  In the tests, I'm going to plot a bar chart of your pivot table.

In [ ]:
%matplotlib inline
answer = None

### SOLUTION
import pandas as pd
import requests
from io import StringIO

# Function to load CSV using requests
def load_csv(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return pd.read_csv(StringIO(response.text), low_memory=False)

# Load the NPI data
npi_data = load_csv('https://hds5210-data.s3.amazonaws.com/npidata.csv')

# Load the CMS data for each year
cms_2014 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2014.csv')
cms_2015 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2015.csv')
cms_2016 = load_csv('https://hds5210-data.s3.amazonaws.com/cms2016.csv')

# Add a new column for the year
cms_2014['year'] = 2014
cms_2015['year'] = 2015
cms_2016['year'] = 2016

# Concatenate the DataFrames into a single DataFrame
cms_combined = pd.concat([cms_2014, cms_2015, cms_2016], ignore_index=True)

# Rename the NPI column if necessary
if 'NPI' in npi_data.columns:
    npi_data.rename(columns={'NPI': 'npi'}, inplace=True)

# Ensure the 'npi' columns are of the same type
npi_data['npi'] = npi_data['npi'].astype(str)
cms_combined['npi'] = cms_combined['npi'].astype(str)

# Merge the CMS data with the NPI data
answer = pd.merge(cms_combined, npi_data, on='npi', how='left')

# Create the 'State' column
answer['State'] = answer['Provider Business Mailing Address State Name'].fillna('XX')

# Create the pivot table
# Assuming 'total_services' is the column that represents the total services
# If the column name is different, replace 'total_services' with the correct name
answer_pivot = answer.pivot_table(index='State', columns='year', values='total_services', aggfunc='sum', fill_value=0)

# Assign the resulting pivot table to 'answer'
answer = answer_pivot

# Display the pivot table
print(answer)

year          2014         2015  2016
State                                
XX     168396862.5  174191868.4   0.0


In [ ]:
assert(answer.shape == (4,3))
assert(answer.sum().sum() == 519185664.6999999)
assert(answer[2016].sum() == 176596933.80000004)
assert(answer.loc['WY'].sum() == 10892707.4)

AssertionError: 

In [ ]:
%matplotlib inline
answer.plot.bar()